In [1]:
#en esta funcion vamos a realizar una seleccion de HVG variable por el numero de genes por data set, además de una noramalización y escalado utilizando scanpy
import os
import anndata as ad
import scanpy as sc
import numpy as np
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp

logging.basicConfig(filename='processing_errors.log', level=logging.ERROR)


def plot_count_distribution(adata, title):
    plt.figure(figsize=(10, 5))
    sc.pl.highest_expr_genes(adata, n_top=20, show=False)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(f"{title.replace(' ', '_')}.png")
    plt.close()

def plot_hvg(adata):
    plt.figure(figsize=(10, 5))
    sc.pl.highly_variable_genes(adata, show=False)
    plt.tight_layout()
    plt.savefig("highly_variable_genes.png")
    plt.close()

def plot_hvg_heatmap(adata):
    plt.figure(figsize=(12, 8))
    sc.pl.heatmap(adata, var_names=adata.var.highly_variable.sort_values(ascending=False).head(50).index, 
                  groupby='source_file', show_gene_labels=True, show=False)
    plt.title("Top 50 Highly Variable Genes")
    plt.tight_layout()
    plt.savefig("hvg_heatmap.png")
    plt.close()

def plot_pca(adata):
    sc.tl.pca(adata)
    plt.figure(figsize=(10, 5))
    sc.pl.pca(adata, color='source_file', show=False)
    plt.tight_layout()
    plt.savefig("pca_plot.png")
    plt.close()

def process_data(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if filename.endswith('.h5ad'):
            input_file = os.path.join(input_dir, filename)
            output_file = os.path.join(output_dir, f"processed_{filename}")
            
            print(f"Procesando archivo: {filename}")
            
            try:
                # Cargar el archivo h5ad
                adata = ad.read_h5ad(input_file)
                print(f"Archivo cargado. Forma inicial: {adata.shape}")
                
                # Verificar integridad de datos
                print(f"Tipo de datos de adata.X: {type(adata.X)}")
                
                # Manejar valores infinitos y NaN
                logging.info("Manejando valores infinitos y NaN...")
                if sp.issparse(adata.X):
                    adata.X = sp.csr_matrix(adata.X)
                    adata.X.data = np.where(np.isinf(adata.X.data) | np.isnan(adata.X.data), 0, adata.X.data)
                else:
                    adata.X = np.where(np.isinf(adata.X) | np.isnan(adata.X), 0, adata.X)
                
                # Visualización de distribución de cuentas iniciales
                plot_count_distribution(adata, f"Distribución de cuentas iniciales - {filename}")
                
                # Hacer los nombres de observaciones únicos
                print("Haciendo los nombres de observaciones únicos...")
                adata.obs_names_make_unique()
                
                # Guardar cuentas 
                print("Guardando cuentas crudas...")
                adata.layers['counts'] = adata.X.copy()
                
                # Filtrar valores infinitos
                adata = adata[:, np.isfinite(adata.X.sum(axis=0))]
                
                # Aplicar transformación logarítmica a los datos crudos
                sc.pp.log1p(adata)
                
                # Selección de genes altamente variables (HVG) en datos transformados
                print("Seleccionando genes altamente variables...")
                n_genes = adata.n_vars
                if n_genes > 20000:
                    n_top_genes = int(n_genes * 0.07)
                elif n_genes > 10000:
                    n_top_genes = 2000
                else:
                    n_top_genes = n_genes
                
                print(f"Número de HVG seleccionados: {n_top_genes}")
                sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes, flavor='seurat', batch_key='source_file')
                
                # Visualización de genes altamente variables
                plot_hvg(adata)
                
                # Filtrar para mantener solo HVG
                adata = adata[:, adata.var.highly_variable]
                print(f"Forma después de seleccionar HVG: {adata.shape}")
                
                # Visualización de heatmap de HVG
                plot_hvg_heatmap(adata)
                
                # Normalización
                print("Realizando normalización...")
                sc.pp.normalize_total(adata, target_sum=1e4)
                
                # Visualización de distribución de cuentas después de normalización
                plot_count_distribution(adata, f"Distribución de cuentas normalizadas - {filename}")
                
                # Escalado
                print("Realizando escalado...")
                sc.pp.scale(adata, max_value=10)
                
                # Visualización PCA después del escalado
                plot_pca(adata)
                
                # Guardar datos procesados en raw
                adata.raw = adata
                
                # Guardar el resultado
                print(f"Guardando resultado en: {output_file}")
                adata.write_h5ad(output_file)
                
                print(f"Procesamiento completado para {filename}")
                print(f"Forma final del objeto AnnData: {adata.shape}")
                
            except Exception as e:
                logging.error(f"Error al procesar {filename}: {str(e)}")
                print(f"Error al procesar {filename}. Ver log para detalles.")
                continue
    
    print("Procesamiento de todos los archivos completado.")

# Uso del script
input_directory = '/app/project/pipeline_colombia/3.doublet_removal'
output_directory = '/app/project/pipeline_colombia/3.5normalized&scaled_by_group'
process_data(input_directory, output_directory)

Procesando archivo: filtered_b cells_data.h5ad
Archivo cargado. Forma inicial: (7352, 2000)
Tipo de datos de adata.X: <class 'scipy.sparse._csr.csr_matrix'>
Haciendo los nombres de observaciones únicos...
Guardando cuentas crudas...
Seleccionando genes altamente variables...
Número de HVG seleccionados: 2000


/root/.pyenv/versions/3.9.7/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:405: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


Forma después de seleccionar HVG: (7352, 2000)


/root/.pyenv/versions/3.9.7/lib/python3.9/site-packages/scanpy/plotting/_utils.py:471: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + "_colors"] = colors_list
/tmp/ipykernel_80498/1226614483.py:34: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Realizando normalización...
Realizando escalado...
Guardando resultado en: /app/project/pipeline_colombia/3.5normalized&scaled_by_group/processed_filtered_b cells_data.h5ad
Procesamiento completado para filtered_b cells_data.h5ad
Forma final del objeto AnnData: (7352, 2000)
Procesando archivo: filtered_nk cells_data.h5ad
Archivo cargado. Forma inicial: (12206, 2000)
Tipo de datos de adata.X: <class 'scipy.sparse._csr.csr_matrix'>
Haciendo los nombres de observaciones únicos...
Guardando cuentas crudas...
Seleccionando genes altamente variables...
Número de HVG seleccionados: 2000


...

<Figure size 1000x500 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>

<Figure size 1000x500 with 0 Axes>